<a href="https://colab.research.google.com/github/xaviorlewis/arete.gallery/blob/main/get_spotify_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spotipy

     |████████████████████████████████| 62 kB 374 kB/s 
     |████████████████████████████████| 138 kB 38.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.26.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
import spotipy
import spotipy.oauth2 as util
import pandas as pd

In [ ]:
#spotify api information
CLIENT_ID = "6106dba10817485b8286c84cfa9489d4"
CLIENT_SECRET = "73cf20eb8aba4f109b310a67d5defac8"

In [ ]:
token = util.SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
cache_token = token.get_access_token(as_dict=False)
sp = spotipy.Spotify(cache_token)

In [ ]:
#create function to scrape spotify api and build dataframe from found data

def analyze_playlist2(user_id, playlist_id):

# Create empty dataframe
  playlist_features_list = ["artist","album","track_name",  "track_id","danceability","energy","key","loudness","mode", "speechiness","instrumentalness","liveness","valence","tempo", "duration_ms","time_signature"]

  playlist_df = pd.DataFrame(columns = playlist_features_list)

# Loop through every track in the playlist, extract features and append the features to the playlist df
  creator = "lstanfill"
  playlist = sp.user_playlist_tracks(creator, playlist_id)["tracks"]["items"]
  for track in playlist:
    # Create empty dict
    playlist_features = {}
    # Get metadata
    playlist_features["artist"] = track["track"]["album"]["artists"][0]["name"]
    playlist_features["album"] = track["track"]["album"]["name"]
    playlist_features["track_name"] = track["track"]["name"]
    playlist_features["track_id"] = track["track"]["id"]
    
# Get audio features
    audio_features = sp.audio_features(playlist_features["track_id"])[0]
    for feature in playlist_features_list[4:]:
        playlist_features[feature] = audio_features[feature]
    
    # Concat the dfs
    track_df = pd.DataFrame(playlist_features, index = [0])
    playlist_df = pd.concat([playlist_df, track_df], ignore_index = True)
    
  return playlist_df

Too many timeout errors forced us to separate the master playlist into 4 smaller ones. Scraping below:

In [ ]:
py1 = analyze_playlist2("lstanfll", "32edHT48Bbq7bHgetgapLo?si=a748190a455b4e18")

In [ ]:
py2 = analyze_playlist2("lstanfll", "204YEv34ZkI1wZCKymK9aI?si=9184fcaa14804dd0")

In [ ]:
py3 = analyze_playlist2("lstanfll", "0Z6sS68aNmWGsMRCes9LDZ?si=2327a18271fd4f6d")

In [ ]:
py4 = analyze_playlist2("lstanfll", "38PokUd0iSIk5MIVqqdU3n?si=b592bba7974646b7")

In [ ]:
#concatenate the four dataframes into one

fullpy = pd.concat([py1,py2,py3,py4], ignore_index=True, sort=False)
fullpy

,artist,album,track_name,track_id,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Bad Bunny,X 100PRE,NI BIEN NI MAL,278kSqsZIiYp8p3QjYAqa8,0.826,0.449,5,-8.330,0,0.0674,1.85e-05,0.2730,0.337,128.028,236216,4
1,Bad Bunny,X 100PRE,200 Mph,5jGUJXnELDaDTgiRjD9lsg,0.850,0.500,8,-6.708,1,0.0665,0.000384,0.0947,0.816,160.020,170509,4
2,Bad Bunny,X 100PRE,Cuando Perriabas,5mj8WVFcKdGA8p9HWGTSLc,0.787,0.705,0,-7.582,1,0.0695,1.06e-06,0.1080,0.499,81.998,188654,4
3,Bad Bunny,X 100PRE,La Romana,1khmgu0pveJbkbpbkyvcQv,0.655,0.725,0,-5.497,1,0.1880,0.00264,0.0611,0.326,125.800,300579,4
4,Bad Bunny,X 100PRE,Como Antes,69ZaPBHhRMRDjRpW1ivnOU,0.767,0.379,0,-10.348,1,0.0385,0.000145,0.2170,0.252,103.956,230578,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372,BTS,Wings,BTS Cypher 4,1QWxddJmOhQb1vDdyTipMR,0.817,0.797,1,-4.395,0,0.0840,7.77e-06,0.0551,0.362,119.977,294463,4
373,BTS,Wings,Am I Wrong,1ORnZ5RmTKhVxPrjORKxAE,0.761,0.834,6,-4.992,1,0.0494,1e-05,0.0868,0.680,107.988,213082,4
374,BTS,Wings,21st Century Girl,6RioKpKPuG1vuyU0wnlAZx,0.716,0.906,2,-3.539,1,0.0717,0,0.0547,0.762,101.028,192597,4
375,BTS,Wings,2! 3!,0ma7lbs8BsVIv2kSaqOsDI,0.439,0.928,7,-4.096,1,0.2330,0,0.1000,0.599,169.928,272537,4


In [ ]:
#save as csv
fullpy.to_csv('/content/drive/MyDrive/QRML/projects/final/data/feature_df.csv')